In [1]:
import pandas
import pytest
from src.energiapy.components.resource import Resource, VaryingResource
from src.energiapy.components.temporal_scale import TemporalScale
from src.energiapy.components.process import Process, VaryingProcess
from src.energiapy.components.location import Location
from src.energiapy.components.scenario import Scenario
from src.energiapy.components.material import Material
from src.energiapy.components.transport import Transport
from src.energiapy.components.network import Network
from src.energiapy.components.case_study import CaseStudy
from src.energiapy.model.formulate import formulate, Constraints, Objective
from src.energiapy.model.solve import solve


In [2]:
capacity_factor = pandas.DataFrame(data=[0.5, 0.75, 1, 0.5])
price_factor = pandas.DataFrame(data=[0.75, 0.8, 0.25, 1.0])
demand_factor = pandas.DataFrame(data=[0.6, 0.8, 0.4, 1.0])

scales = TemporalScale(discretization_list=[1, 4])

resource_certain_availability = Resource(
    name='resource_certain_availability', cons_max=100)

resource_implicit = Resource(name='resource_implicit')

resource_implicit2 = Resource(name='resource_implicit2')

resource_deterministic_demand = Resource(
    name='resource_deterministic_demand', demand=True, revenue=2000)

resource_deterministic_price = Resource(name='resource_deterministic_price', cons_max=100, price=3.00, varying=[
                                        VaryingResource.DETERMINISTIC_PRICE])

resource_sell = Resource(name='resource_sell', sell=True)

process_deterministic_capacity = Process(name='process_deterministic_capacity', conversion={resource_certain_availability: -1.25, resource_implicit: 1},
                                            capex=100, fopex=10, vopex=1, cap_max=100, varying=[VaryingProcess.UNCERTAIN_CAPACITY])

process_certain_capacity = Process(name='process_certain_capacity', conversion={resource_implicit: -0.5, resource_deterministic_price: -0.75, resource_implicit2: 1},
                                    capex=200, fopex=25, vopex=5, cap_max=100)

process_certain_capacity2 = Process(name='process_certain_capacity2', conversion={resource_implicit2: -1, resource_deterministic_demand: 1},
                                    capex=10, fopex=1, vopex=0.1, cap_max=100)

location1 = Location(name='location1', processes={process_deterministic_capacity}, capacity_factor={process_deterministic_capacity: capacity_factor},
                        scales=scales, capacity_factor_scale_level=1)

location2 = Location(name='location2', processes={process_certain_capacity}, price_factor={resource_deterministic_price: price_factor},
                        scales=scales, price_factor_scale_level=1)

location3 = Location(name='location3', processes={process_certain_capacity2}, demand_factor={resource_deterministic_demand: demand_factor}, scales=scales,
                        demand_factor_scale_level=1)
transport = Transport(name='transport', resources={
                        resource_implicit}, trans_max=100, trans_loss=0.1, capex=40, vopex=1)
transport2 = Transport(name='transport2', resources={
                        resource_implicit2}, trans_max=100, trans_loss=0.05, capex=20, vopex=5)

distance_matrix = [
    [10, 0],
    [0, 50],
]

transport_matrix = [
    [[transport], []],
    [[], [transport2]],
]

network = Network(name='network', source_locations=[location1, location2], sink_locations=[location2, location3],
                    distance_matrix=distance_matrix, transport_matrix=transport_matrix, scales=scales)

scenario = Scenario(name='scenario', network=network, scales=scales, scheduling_scale_level=1,
                    network_scale_level=0, demand_scale_level=1, demand={l: {resource_deterministic_demand: 50} if l == location3 else {resource_deterministic_demand: 0} for l in network.locations})
casestudy = CaseStudy(name='casestudy', scenarios=[scenario])

casestudy.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                    Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.TRANSPORT}, objective=Objective.COST)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint network cost
constraint production mode
constraint inventory balance
constraint inventory network
constraint resource export
constraint transport export
constraint export
constraint transport capex
constraint transport network capex
constraint transport export network
constraint transport vopex
constraint transport network vopex
constraint transport fopex
constraint transport network fopex
constraint transport capacity UB no bin
constraint transport capacity LB no bin
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint transport capacity UB
constraint transport capacity LB
objective cost
constraint demand


{'scenario': <pyomo.core.base.PyomoModel.ConcreteModel at 0x1ff2e804c20>}

In [3]:
results = casestudy.solve(solver = 'gurobi')

Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 309 rows, 340 columns and 586 nonzeros
Model fingerprint: 0xd348a157
Variable types: 323 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 1e+02]
Presolve removed 309 rows and 340 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 108530 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.085303750000e+05, best bound 1.085303750000e+05, gap 0.0000%


In [4]:
import copy 
def round_values(d):
    if isinstance(d, dict):
        for key, value in d.items():
            if isinstance(value, (float, int)):
                # Round the numerical value to 3 decimal places
                d[key] = round(value, 3)
            elif isinstance(value, dict):
                round_values(value)
    elif isinstance(d, list):
        for i, item in enumerate(d):
            if isinstance(item, (float, int)):
                # Round the numerical value to 3 decimal places
                d[i] = round(item, 3)
            elif isinstance(item, (dict, list)):
                round_values(item)

In [5]:
res = copy.deepcopy(results.results['scenario'].output)

In [6]:
round_values(res)

In [7]:
res

{'termination': 'optimal',
 'LB': 108530.375,
 'UB': 108530.375,
 'n_cons': 309,
 'n_vars': 340,
 'n_binvars': 17,
 'n_intvars': 17,
 'n_convars': 306,
 'n_nonzero': 586,
 'P': {('location3', 'process_certain_capacity2', 0, 0): 30.0,
  ('location3', 'process_certain_capacity2', 0, 1): 40.0,
  ('location3', 'process_certain_capacity2', 0, 2): 20.0,
  ('location3', 'process_certain_capacity2', 0, 3): 50.0,
  ('location2', 'process_certain_capacity', 0, 0): 30.0,
  ('location2', 'process_certain_capacity', 0, 1): 40.0,
  ('location2', 'process_certain_capacity', 0, 2): 20.0,
  ('location2', 'process_certain_capacity', 0, 3): 50.0,
  ('location1', 'process_deterministic_capacity', 0, 0): 15.0,
  ('location1', 'process_deterministic_capacity', 0, 1): 20.0,
  ('location1', 'process_deterministic_capacity', 0, 2): 10.0,
  ('location1', 'process_deterministic_capacity', 0, 3): 25.0},
 'C': {('location2', 'resource_deterministic_price', 0, 0): 22.5,
  ('location2', 'resource_deterministic_price

In [2]:
capacity_factor = pandas.DataFrame(data=[0.5, 0.75, 1, 0.5])
price_factor = pandas.DataFrame(data=[0.75, 0.8, 0.25, 1.0])
demand_factor = pandas.DataFrame(data=[0.6, 0.8, 0.4, 1.0])

scales = TemporalScale(discretization_list=[1, 4])

resource_certain_availability = Resource(
    name='resource_certain_availability', cons_max=100)

resource_implicit = Resource(name='resource_implicit')

resource_implicit2 = Resource(name='resource_implicit2')

resource_deterministic_demand = Resource(
    name='resource_deterministic_demand', demand=True, revenue=2000)

resource_deterministic_price = Resource(name='resource_deterministic_price', cons_max=100, price=3.00, varying=[
                                        VaryingResource.DETERMINISTIC_PRICE])

resource_sell = Resource(name='resource_sell', sell=True)

process_deterministic_capacity = Process(name='process_deterministic_capacity', conversion={resource_certain_availability: -1.25, resource_implicit: 1},
                                            capex=100, fopex=10, vopex=1, cap_max=100, varying=[VaryingProcess.UNCERTAIN_CAPACITY])

process_certain_capacity = Process(name='process_certain_capacity', conversion={resource_implicit: -0.5, resource_deterministic_price: -0.75, resource_implicit2: 1},
                                    capex=200, fopex=25, vopex=5, cap_max=100)

process_certain_capacity2 = Process(name='process_certain_capacity2', conversion={resource_implicit2: -1, resource_deterministic_demand: 1},
                                    capex=10, fopex=1, vopex=0.1, cap_max=100)

location1 = Location(name='location1', processes={process_deterministic_capacity}, capacity_factor={process_deterministic_capacity: capacity_factor},
                        scales=scales, demand_scale_level=1, capacity_scale_level=1, price_scale_level=1)

location2 = Location(name='location2', processes={process_certain_capacity}, price_factor={resource_deterministic_price: price_factor},
                        scales=scales, demand_scale_level=1, capacity_scale_level=1, price_scale_level=1)

location3 = Location(name='location3', processes={process_certain_capacity2}, demand_factor={resource_deterministic_demand: demand_factor}, scales=scales,
                        demand_scale_level=1, capacity_scale_level=1, price_scale_level=1)
transport = Transport(name='transport', resources={
                        resource_implicit}, trans_max=100, trans_loss=0.1, capex=40, vopex=1)
transport2 = Transport(name='transport2', resources={
                        resource_implicit2}, trans_max=100, trans_loss=0.05, capex=20, vopex=5)

distance_matrix = [
    [10, 0],
    [0, 50],
]

transport_matrix = [
    [[transport], []],
    [[], [transport2]],
]

network = Network(name='network', source_locations=[location1, location2], sink_locations=[location2, location3],
                    distance_matrix=distance_matrix, transport_matrix=transport_matrix, scales=scales)

# scenario = Scenario(name='scenario', network=network, scales=scales, scheduling_scale_level=1,
#                     network_scale_level=0, demand_scale_level=1, purchase_scale_level=1, capacity_scale_level=1, demand={l: {resource_deterministic_demand: 50} if l == location3 else {resource_deterministic_demand: 0} for l in network.locations})

scenario = Scenario(name='scenario', network=network, scales=scales, scheduling_scale_level=1,
                    network_scale_level=0, demand_scale_level=1, purchase_scale_level=1, capacity_scale_level=1, demand={location3: {resource_deterministic_demand: 50}})


casestudy = CaseStudy(name='casestudy', scenarios=[scenario])

casestudy.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                    Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.TRANSPORT}, objective=Objective.COST)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint network cost
constraint production mode
constraint inventory balance
constraint inventory network
constraint resource export
constraint transport export
constraint export
constraint transport capex
constraint transport network capex
constraint transport export network
constraint transport vopex
constraint transport network vopex
constraint transport fopex
constraint transport network fopex
constraint transport capacity UB no bin
constraint transport capacity LB no bin
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint transport capacity UB
constraint transport capacity LB
constraint demand
objective cost


{'scenario': <pyomo.core.base.PyomoModel.ConcreteModel at 0x27622892f20>}

In [3]:
results = casestudy.solve(solver='gurobi')

Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 309 rows, 348 columns and 586 nonzeros
Model fingerprint: 0x1aee725e
Variable types: 331 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 1e+02]
Presolve removed 309 rows and 348 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 108530 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.085303750000e+05, best bound 1.085303750000e+05, gap 0.0000%


In [4]:
casestudy.formulations['scenario'].constraint_demand.pprint()

constraint_demand : specific demand for resources
    Size=4, Index=constraint_demand_index, Active=True
    Key                                                  : Lower : Body                                           : Upper : Active
    ('location3', 'resource_deterministic_demand', 0, 0) :  30.0 : S[location3,resource_deterministic_demand,0,0] :  +Inf :   True
    ('location3', 'resource_deterministic_demand', 0, 1) :  40.0 : S[location3,resource_deterministic_demand,0,1] :  +Inf :   True
    ('location3', 'resource_deterministic_demand', 0, 2) :  20.0 : S[location3,resource_deterministic_demand,0,2] :  +Inf :   True
    ('location3', 'resource_deterministic_demand', 0, 3) :  50.0 : S[location3,resource_deterministic_demand,0,3] :  +Inf :   True


In [5]:
casestudy.formulations['scenario'].P.pprint()

P : Production
    Size=12, Index=P_index
    Key                                                   : Lower : Value : Upper : Fixed : Stale : Domain
    ('location1', 'process_deterministic_capacity', 0, 0) :     0 :  15.0 :  None : False : False : NonNegativeReals
    ('location1', 'process_deterministic_capacity', 0, 1) :     0 :  20.0 :  None : False : False : NonNegativeReals
    ('location1', 'process_deterministic_capacity', 0, 2) :     0 :  10.0 :  None : False : False : NonNegativeReals
    ('location1', 'process_deterministic_capacity', 0, 3) :     0 :  25.0 :  None : False : False : NonNegativeReals
          ('location2', 'process_certain_capacity', 0, 0) :     0 :  30.0 :  None : False : False : NonNegativeReals
          ('location2', 'process_certain_capacity', 0, 1) :     0 :  40.0 :  None : False : False : NonNegativeReals
          ('location2', 'process_certain_capacity', 0, 2) :     0 :  20.0 :  None : False : False : NonNegativeReals
          ('location2', 'process

In [6]:
casestudy.formulations['scenario'].constraint_demand.pprint()

constraint_demand : specific demand for resources
    Size=4, Index=constraint_demand_index, Active=True
    Key                                                  : Lower : Body                                           : Upper : Active
    ('location3', 'resource_deterministic_demand', 0, 0) :  30.0 : S[location3,resource_deterministic_demand,0,0] :  +Inf :   True
    ('location3', 'resource_deterministic_demand', 0, 1) :  40.0 : S[location3,resource_deterministic_demand,0,1] :  +Inf :   True
    ('location3', 'resource_deterministic_demand', 0, 2) :  20.0 : S[location3,resource_deterministic_demand,0,2] :  +Inf :   True
    ('location3', 'resource_deterministic_demand', 0, 3) :  50.0 : S[location3,resource_deterministic_demand,0,3] :  +Inf :   True


In [7]:
casestudy.formulations['scenario'].constraint_resource_consumption_certain.pprint()

constraint_resource_consumption_certain : restricts resource consumption to certain availablity
    Size=8, Index=constraint_resource_consumption_certain_index, Active=True
    Key                                                  : Lower : Body                                           : Upper : Active
    ('location1', 'resource_certain_availability', 0, 0) :  -Inf : C[location1,resource_certain_availability,0,0] : 100.0 :   True
    ('location1', 'resource_certain_availability', 0, 1) :  -Inf : C[location1,resource_certain_availability,0,1] : 100.0 :   True
    ('location1', 'resource_certain_availability', 0, 2) :  -Inf : C[location1,resource_certain_availability,0,2] : 100.0 :   True
    ('location1', 'resource_certain_availability', 0, 3) :  -Inf : C[location1,resource_certain_availability,0,3] : 100.0 :   True
     ('location2', 'resource_deterministic_price', 0, 0) :  -Inf :  C[location2,resource_deterministic_price,0,0] : 100.0 :   True
     ('location2', 'resource_deterministi

In [8]:
scenario.location_process_dict

{'location1': {'process_deterministic_capacity'},
 'location3': {'process_certain_capacity2'},
 'location2': {'process_certain_capacity'}}

In [9]:
casestudy.formulations['scenario'].constraint_inventory_balance.pprint()

constraint_inventory_balance : mass balance across scheduling scale
    Size=28, Index=constraint_inventory_balance_index, Active=True
    Key                                                  : Lower : Body                                                                                                                                                        : Upper : Active
    ('location1', 'resource_certain_availability', 0, 0) :   0.0 :                                                   C[location1,resource_certain_availability,0,0] - 1.25*P_m[location1,process_deterministic_capacity,0,0,0] :   0.0 :   True
    ('location1', 'resource_certain_availability', 0, 1) :   0.0 :                                                   C[location1,resource_certain_availability,0,1] - 1.25*P_m[location1,process_deterministic_capacity,0,0,1] :   0.0 :   True
    ('location1', 'resource_certain_availability', 0, 2) :   0.0 :                                                   C[location1,resource_certain

In [10]:
"""LP for scenario with variability"""
demand_factor = pandas.DataFrame(data={'Value': [0.5, 1, 0.5, 0.25]})
capacity_factor = pandas.DataFrame(data={'Value': [1, 0.5, 1, 0.25]})
availability_factor = pandas.DataFrame(data={'Value': [1, 1, 0.5, 0.5]})
price_factor = pandas.DataFrame(data={'Value': [0.25, 1, 0.5, 0.75]})
revenue_factor = pandas.DataFrame(data={'Value': [0.25, 1, 0.25, 0.5]})

scales = TemporalScale(discretization_list=[1, 4])

Resource_certain_availability = Resource(
    name='resource_certain_availability', cons_max=100, price=2)

Resource_deterministic_availability = Resource(name='resource_deterministic_availability', cons_max=100, price=5, varying=[
    VaryingResource.DETERMINISTIC_AVAILABILITY])
Resource_deterministic_demand = Resource(name='resource_deterministic_demand', demand=True, varying=[
    VaryingResource.DETERMINISTIC_DEMAND])
Resource_deterministic_price = Resource(name='resource_deterministic_price', cons_max=100, price=10, varying=[
                                        VaryingResource.DETERMINISTIC_PRICE])
Resource_deterministic_revenue = Resource(name='resource_deterministic_revenue', demand=True, revenue=500, varying=[
    VaryingResource.DETERMINISTIC_REVENUE])

Process_deterministic_capacity = Process(name='process_deterministic_capacity', conversion={Resource_certain_availability: -1, Resource_deterministic_demand: 1},
                                            capex=1000, fopex=10, vopex=1, cap_max=100, cap_min=0, varying=[VaryingProcess.DETERMINISTIC_CAPACITY])
Process_certain_capacity = Process(name='process_certain_capacity', conversion={
    Resource_deterministic_demand: -0.5, Resource_deterministic_availability: -0.5, Resource_deterministic_price: -0.25, Resource_deterministic_revenue: 1},
    capex=100, fopex=10, vopex=1, cap_max=100, cap_min=0)
Process_storage = Process(name='process_storage', storage=Resource_deterministic_demand,
                            capex=100, fopex=5, vopex=0.5, cap_max=50, cap_min=0, store_max=25, storage_cost=0.1)

Location1 = Location(name='location', processes={Process_certain_capacity, Process_deterministic_capacity, Process_storage}, scales=scales,
                        demand_scale_level=1, capacity_scale_level=1, price_scale_level=1, availability_scale_level=1,
                        demand_factor={Resource_deterministic_demand: demand_factor}, capacity_factor={Process_deterministic_capacity: capacity_factor},
                        price_factor={Resource_deterministic_price: price_factor}, availability_factor={Resource_deterministic_availability: availability_factor},
                        revenue_factor={Resource_deterministic_revenue: revenue_factor})

Scenario1 = Scenario(name='scenario', scales=scales, network=Location1, demand_scale_level=1, network_scale_level=0, scheduling_scale_level=1,
                        capacity_scale_level=1, purchase_scale_level=1, availability_scale_level=1,
                        demand={Location1: {Resource_deterministic_demand: 25, Resource_deterministic_revenue: 25}})

case_study = formulate(scenario=Scenario1, constraints={Constraints.COST, Constraints.INVENTORY,
                                                    Constraints.PRODUCTION, Constraints.RESOURCE_BALANCE, Constraints.DEMAND}, objective=Objective.COST)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint network cost
constraint production mode
constraint inventory balance
constraint inventory network
constraint demand
objective cost


In [11]:
results = solve(scenario=Scenario1,
                instance= case_study, solver='gurobi', name='LP')

Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 155 rows, 138 columns and 350 nonzeros
Model fingerprint: 0x26cf19a7
Coefficient statistics:
  Matrix range     [1e-01, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+02]
Presolve removed 140 rows and 122 columns
Presolve time: 0.00s
Presolved: 15 rows, 16 columns, 39 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8554912e+04   1.015950e+01   0.000000e+00      0s
       9    4.7423333e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.742333333e+04


In [ ]:
case_study.constraint_demand.pprint()

constraint_demand : specific demand for resources
    Size=8, Index=constraint_demand_index, Active=True
    Key                                                  : Lower : Body                                           : Upper : Active
     ('location', 'resource_deterministic_demand', 0, 0) :  12.5 :  S[location,resource_deterministic_demand,0,0] :  +Inf :   True
     ('location', 'resource_deterministic_demand', 0, 1) :  25.0 :  S[location,resource_deterministic_demand,0,1] :  +Inf :   True
     ('location', 'resource_deterministic_demand', 0, 2) :  12.5 :  S[location,resource_deterministic_demand,0,2] :  +Inf :   True
     ('location', 'resource_deterministic_demand', 0, 3) :  6.25 :  S[location,resource_deterministic_demand,0,3] :  +Inf :   True
    ('location', 'resource_deterministic_revenue', 0, 0) :  25.0 : S[location,resource_deterministic_revenue,0,0] :  +Inf :   True
    ('location', 'resource_deterministic_revenue', 0, 1) :  25.0 : S[location,resource_deterministic_revenue,

In [ ]:
case_study.constraint_location_discharge.pprint()

constraint_location_discharge : sums up discharge of resource over the temporal scale at location
    Size=2, Index=constraint_location_discharge_index, Active=True
    Key                                               : Lower : Body                                                                                                                                                                                                                                                        : Upper : Active
     ('location', 'resource_deterministic_demand', 0) :   0.0 :      S_location[location,resource_deterministic_demand,0] - (S[location,resource_deterministic_demand,0,0] + S[location,resource_deterministic_demand,0,1] + S[location,resource_deterministic_demand,0,2] + S[location,resource_deterministic_demand,0,3]) :   0.0 :   True
    ('location', 'resource_deterministic_revenue', 0) :   0.0 : S_location[location,resource_deterministic_revenue,0] - (S[location,resource_deterministic_revenue,0,0] +

In [ ]:
case_study.S.pprint()

S : Resource Dispensed/Sold
    Size=8, Index=S_index
    Key                                                  : Lower : Value : Upper : Fixed : Stale : Domain
     ('location', 'resource_deterministic_demand', 0, 0) :     0 :  12.5 :  None : False : False : NonNegativeReals
     ('location', 'resource_deterministic_demand', 0, 1) :     0 :  25.0 :  None : False : False : NonNegativeReals
     ('location', 'resource_deterministic_demand', 0, 2) :     0 :  12.5 :  None : False : False : NonNegativeReals
     ('location', 'resource_deterministic_demand', 0, 3) :     0 :  6.25 :  None : False : False : NonNegativeReals
    ('location', 'resource_deterministic_revenue', 0, 0) :     0 :  25.0 :  None : False : False : NonNegativeReals
    ('location', 'resource_deterministic_revenue', 0, 1) :     0 :  25.0 :  None : False : False : NonNegativeReals
    ('location', 'resource_deterministic_revenue', 0, 2) :     0 :  25.0 :  None : False : False : NonNegativeReals
    ('location', 'resource_d